In [1]:
import csv
import os
import pandas as pd
from shutil import copy #file
from os import listdir

csv.field_size_limit(300000) # Some records are too large and require a larger field_size_limit

131072

In [2]:
def addSeed(csv_in, csv_out, keywords, biases, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('Keywords:', keywords)
    print('Biases:', biases)
    print('Depth:', depth)
    
    with open ("model/plotting/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("model/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if keywords==[]:
                print('No keywords')

                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1
 
                    if (row[17] in biases or (row[17]=="" and row[14] in biases)) and int(row[3])==depth:
                    #if row[16] in biases and int(row[3])==depth and row[2]=='andrew cuomo':
                        #row.append(0)
                        csv_writer.writerow(row)
            elif biases==[]:
                print('No biasess')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[0] in keywords and int(row[3])==depth:
                        #row.append(0)
                        csv_writer.writerow(row)
            else:
                print('Keywords and biasess')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if (row[17] in biases or (row[17]=="" and row[14] in biases)) and row[0] in keywords and int(row[3])==depth:
                        #row.append(0)
                        csv_writer.writerow(row)

In [19]:
def addChildren(csv_in, csv_seed, csv_out, depth):
    count=0
    next_depth=depth + 1
    print('Depth:', depth, csv_seed)

    df_seed=pd.read_csv("model/plotting/" + csv_seed)
    videos=set(df_seed[df_seed['current_depth']==depth]['video'].values)
    keywords=set(df_seed[df_seed['current_depth']==depth]['keyword'].values)
    traces=set(df_seed[df_seed['current_depth']==depth]['trace'].values)

    print(len(videos))

    with open ("model/plotting/" + csv_out, 'a') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("model/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            next(csv_reader, None)

            for row in csv_reader:
                if count % 150000 == 0:
                    print("Read: " + str(count))
                count += 1
                if row[0] in keywords:
                    if int(row[3])==next_depth:
                        if int(row[13]) in traces:
                            keyword=row[0].replace(" ", "")
                            csv_lookup_seed=keyword + "_" + str(depth) + "_" + csv_seed
                            with open("model/plotting/" + csv_lookup_seed, 'r') as seed_read_obj:
                                csv_seed_reader = csv.reader(seed_read_obj)
                                next(csv_seed_reader, None)
                                for seed_row in csv_seed_reader:
                                    #if int(seed_row[3])==depth:
                                    if int(seed_row[12])==int(row[13]) and seed_row[1]==row[2]:
                                        csv_writer.writerow(row)

In [4]:
def add1Child(csv_in, csv_seed, csv_out, depth):
    count=0
    next_depth=depth + 1
    print('Depth:', depth, csv_seed)

    df_seed=pd.read_csv("model/plotting/" + csv_seed)
    videos=set(df_seed[df_seed['current_depth']==depth]['video'].values)
    keywords=set(df_seed[df_seed['current_depth']==depth]['keyword'].values)
    traces=set(df_seed[df_seed['current_depth']==depth]['trace'].values)

    print(len(videos))

    with open ("model/plotting/" + csv_out, 'a') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("model/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            next(csv_reader, None)

            for row in csv_reader:
                if count % 150000 == 0:
                    print("Read: " + str(count))
                count += 1
                if int(row[3])==next_depth:
                    if int(row[13]) in traces:
                        if row[0] in keywords:
                            keyword=row[0].replace(" ", "")
                            csv_lookup_seed=keyword + "_" + str(depth) + "_" + csv_seed
                            with open("model/plotting/" + csv_lookup_seed, 'r') as seed_read_obj:
                                csv_seed_reader = csv.reader(seed_read_obj)
                                next(csv_seed_reader, None)
                                for seed_row in csv_seed_reader:
                                    #if int(seed_row[3])==depth:
                                    if int(seed_row[12])==int(row[13]):
                                        csv_writer.writerow(row)

In [5]:
def splitSeedFile(csv_in, depth):
    count=0
    last_id=0
    print("Splitting depth ", depth)
    with open("model/plotting/" + csv_in, 'r') as read_obj:
        csv_reader = csv.reader(read_obj)
        #next(csv_reader, None)

        for row in csv_reader:
            #print("Splitting: " + str(count))
            count += 1
            if row[3]=='current_depth':
                header_row=row
                continue
            if int(row[3])==depth:
                keyword=row[0].replace(" ", "")
                csv_out=keyword + "_" + str(depth) + "_" + csv_in

                if not os.path.exists('model/plotting/' + csv_out):
                    with open ("model/plotting/" + csv_out, 'w') as write_obj:
                        csv_writer = csv.writer(write_obj)
                        #csv_writer.writerow(('X', 'video', 'keyword', 'current_depth', 'fetch_time', 'parent_video', 'positive', 'negative', 'sentiment', 'sentiment_ratio', 'lgpp', 'nrc_el', 'topic', 'Flesch', 'ARI', 'Flesch.Kincaid', 'Bias', 'sentiment_ratio_plot', 'flech_level', 'ARI_level', 'flesch_kincaid_level', 'parentVideoRow0'))                    
                        csv_writer.writerow(header_row)                    

                if last_id==0:
                    with open ("model/plotting/" + csv_out, 'a') as write_obj:
                        csv_writer = csv.writer(write_obj)
                        #if keyword=='TedCruz':
                            #print('if', row[0], row[3], count)
                        csv_writer.writerow(row)
                        previous_row=row
                        last_id=int(row[12])
                elif row[12]!=last_id:
                    #print('Second', row[0], csv_out)
                    with open ("model/plotting/" + csv_out, 'a') as write_obj:
                        #if keyword=='TedCruz':
                            #print('else', row[0], row[3], count)
                        csv_writer = csv.writer(write_obj)
                        csv_writer.writerow(row)
                        last_id=int(row[12])

In [6]:
def addSeedlgpp(csv_in, csv_out, lgpps, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('lgpp:', lgpps)
    print('Depth:', depth)
    
    with open ("model/plotting/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("model/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if lgpps==[]:
                print('No keywords')

            else:
                print('With lgpp')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[63] in lgpps and int(row[3])==depth:
                        csv_writer.writerow(row)

In [7]:
def addSeednrc_el(csv_in, csv_out, nrc_els, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('nrc_el:', nrc_els)
    print('Depth:', depth)
    
    with open ("model/plotting/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("model/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if nrc_els==[]:
                print('No nrc_els')

            else:
                print('With nrc_els')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[74] in nrc_els and int(row[3])==depth:
                        csv_writer.writerow(row)

In [8]:
def addSeedlm(csv_in, csv_out, lms, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('lms:', lms)
    print('Depth:', depth)
    
    with open ("Subtitles/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("Subtitles/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if lms==[]:
                print('No lms')
            else:
                print('With lms')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[53] in lms and int(row[3])==depth:
                        csv_writer.writerow(row)

In [9]:
def addSeedhuliu(csv_in, csv_out, hulius, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('hulius:', hulius)
    print('Depth:', depth)
    
    with open ("Subtitles/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("Subtitles/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if hulius==[]:
                print('No hulius')

            else:
                print('With hulius')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[42] in hulius and int(row[3])==depth:
                        csv_writer.writerow(row)

In [10]:
def addSeedgeninqposneg(csv_in, csv_out, geninqposnegs, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('geninqposnegs:', geninqposnegs)
    print('Depth:', depth)
    
    with open ("Subtitles/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("Subtitles/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if geninqposnegs==[]:
                print('No geninqposnegs')

            else:
                print('With geninqposnegs')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[42] in geninqposnegs and int(row[3])==depth:
                        csv_writer.writerow(row)

In [11]:
def addSeedafinn(csv_in, csv_out, afinns, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('afinns:', afiafinnsnns)
    print('Depth:', depth)
    
    with open ("Subtitles/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("Subtitles/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if afinns==[]:
                print('No afinns')

            else:
                print('With afinns')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[32] in afinns and int(row[3])==depth:
                        csv_writer.writerow(row)

In [12]:
def addSeedlsd(csv_in, csv_out, lsds, depth):
    print('Createing seed file:', csv_out)
    count=0
    print('lsds:', lsds)
    print('Depth:', depth)
    
    with open ("Subtitles/" + csv_out, 'w') as write_obj:
        csv_writer = csv.writer(write_obj)

        with open("Subtitles/" + csv_in, 'r') as read_obj:
            csv_reader = csv.reader(read_obj)
            row=next(csv_reader)
            row.append('parentVideoRow0')
            csv_writer.writerow(row)

            if lsds==[]:
                print('No lsds')

            else:
                print('With lsds')
                for row in csv_reader:
                    if count % 150000 == 0:
                        print("Read: " + str(count))
                    count += 1

                    if row[27] in lsds and int(row[3])==depth:
                        csv_writer.writerow(row)

In [13]:
csv_in = "classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv"

In [14]:
keyword_prefix = 'Center'
keywords=[]
biases=['C']
depth=0

csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Center_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['C']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Center_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
25
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Center_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
107
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Center_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
432
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Center_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
1433
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Center_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
4345
Read: 0
Read: 150000
Read: 300000
Read: 450000


In [15]:
keyword_prefix = 'Right'
keywords=[]
biases=['R']
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Right_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['R']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Right_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
10
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
38
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
139
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
432
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
1386
Read: 0
Read: 150000
Read: 300000
Read: 450000


In [16]:
keyword_prefix = 'Center-Right'
keywords=[]
biases=['CR', 'RC']
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Center-Right_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['CR', 'RC']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Center-Right_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
19
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
72
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
278
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
900
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
2789
Read: 0
Read: 150000
Read: 300000


In [17]:
keyword_prefix = 'Right-Center-Right'
keywords=[]
biases=['R', 'CR', 'RC']
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Right-Center-Right_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['R', 'CR', 'RC']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Right-Center-Right_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
29
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Right-Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
105
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Right-Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
377
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Right-Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
1159
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Right-Center-Right_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.

In [20]:
keyword_prefix = 'Center-Left'
keywords=[]
biases=['CL', 'LC']
depth=0

#csv_in = "videos_for_clustering2021-06-25_13_14_45.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Center-Left_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['CL', 'LC']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Center-Left_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
61
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
217
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
867
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
2935
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
9013
Read: 0
Read: 150000
Read: 300000
Read

In [21]:
keyword_prefix = 'Left-Center-Left'
keywords=[]
biases=['L', 'CL', 'LC']
depth=0

#csv_in = "videos_for_clustering2021-06-25_13_14_45.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Left-Center-Left_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['L', 'CL', 'LC']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Left-Center-Left_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
72
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Left-Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
255
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Left-Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
1000
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Left-Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
3356
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Left-Center-Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
10221
R

In [ ]:
keyword_prefix = 'Left'
keywords=[]
biases=['L']
depth=0

#csv_in = "videos_for_clustering2021-06-25_13_14_45.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeed(csv_in, csv_out0, keywords, biases, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

Createing seed file: Left_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
Keywords: []
Biases: ['L']
Depth: 0
No keywords
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  0
Depth: 0 Left_0_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
11
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  1
Depth: 1 Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
49
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  2
Depth: 2 Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
207
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  3
Depth: 3 Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
726
Read: 0
Read: 150000
Read: 300000
Read: 450000
Splitting depth  4
Depth: 4 Left_classified_videos_proportional_Dataset5_top152022-01-28_18_36_39.csv
2425
Read: 0
Read: 150000


In [ ]:
keyword_prefix = "lgpp_economy"
keywords=["economy"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lgpp_groups"
keywords=["groups"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lgpp_institutions"
keywords=["institutions"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lgpp_law_and_order"
keywords=["law_and_order"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lgpp_rural"
keywords=["rural"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lgpp_urban"
keywords=["urban"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lgpp_values"
keywords=["values"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlgpp(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_elanger"
keywords=["anger"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_anticipation"
keywords=["anticipation"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_disgust"
keywords=["disgust"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_fear"
keywords=["fear"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_joy"
keywords=["joy"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_negative"
keywords=["negative"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_positive"
keywords=["positive"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_sadness"
keywords=["sadness"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_surprise"
keywords=["surprise"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "nrc_el_trust"
keywords=["trust"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeednrc_el(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_negative"
keywords=["negative"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_positive"
keywords=["positive"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_uncertainty"
keywords=["uncertainty"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_litigious"
keywords=["litigious"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_constraining"
keywords=["constraining"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_superfluous"
keywords=["superfluous"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lm_interesting"
keywords=["interesting"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlm(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "huliu_positive"
keywords=["positive"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedhuliu(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "huliu_negative"
keywords=["negative"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedhuliu(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lsd_negative"
keywords=["negative"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlsd(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "lsd_negative"
keywords=["negative"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedlsd(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "geninqposneg_negative"
keywords=["negative"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedgeninqposneg(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)

In [ ]:
keyword_prefix = "geninqposneg_positive"
keywords=["positive"]
depth=0

#csv_in = "videos_for_clustering2021-06-12_21_01_20.csv"
csv_out0 = keyword_prefix + "_0_" + csv_in
csv_out = keyword_prefix + "_" + csv_in

addSeedgeninqposneg(csv_in, csv_out0, keywords, depth)
copy("model/plotting/" + csv_out0, "model/plotting/" + csv_out)

splitSeedFile(csv_out0, 0)
addChildren(csv_in, csv_out0, csv_out, 0)
splitSeedFile(csv_out, 1)
addChildren(csv_in, csv_out, csv_out, 1)
splitSeedFile(csv_out, 2)
addChildren(csv_in, csv_out, csv_out, 2)
splitSeedFile(csv_out, 3)
addChildren(csv_in, csv_out, csv_out, 3)
splitSeedFile(csv_out, 4)
addChildren(csv_in, csv_out, csv_out, 4)